**Troubleshooting Scripts - TempDB**

Dmitri V. Korotkevitch (MCM, MVP)

email: [dk@aboutsqlserver.com](mailto:dk@aboutsqlserver.com)      blog: [https://aboutsqlserver.com](https://aboutsqlserver.com/) code: [https://github.com/aboutsqlserver/code](https://github.com/aboutsqlserver/code)

SQL Server Advanced Troubleshooting and Performance Tuning (O'Reilly, 2022)      ISBN: 978-1098101923

**tempdb Usage Iinformation**

In [ ]:
SELECT
    CONVERT(DECIMAL(12,3),SUM(user_object_reserved_page_count) / 128.) 
        AS [User Objects (MB)]
    ,CONVERT(DECIMAL(12,3),SUM(internal_object_reserved_page_count) / 128.) 
        AS [Internal Objects (MB)]
    ,CONVERT(DECIMAL(12,3),SUM(version_store_reserved_page_count) / 128.) 
        AS [Version Store (MB)]
    ,CONVERT(DECIMAL(12,3),SUM(unallocated_extent_page_count) / 128.) 
        AS [Free Space (MB)]
FROM
    tempdb.sys.dm_db_file_space_usage WITH (NOLOCK)
OPTION (MAXDOP 1, RECOMPILE);

**tempdb Database File Space Usage**

Need to run in tempdb context

In [ ]:
USE [tempdb]
GO

SELECT 
	file_id
	,type_desc
	,name
	,physical_name
	,state_desc
	,CONVERT(DECIMAL(15,3),size / 128.) AS size_mb
	,CONVERT(DECIMAL(15,3),CONVERT(INT,FILEPROPERTY(name, 'SpaceUsed')) / 128.) AS space_used_mb
    ,CONVERT(DECIMAL(15,3),size - CONVERT(INT,FILEPROPERTY(name, 'SpaceUsed')) / 128.) AS free_space_mb
	,IIF(max_size = -1, NULL, CONVERT(DECIMAL(15,3),max_size / 128.)) AS max_size_mb
FROM 
    tempdb.sys.database_files
ORDER BY
	[type] DESC, file_id

**tempdb Usage Per Session**

In [ ]:
;WITH SpaceUsagePages
AS
(
    SELECT
        ss.session_id
        ,ss.user_objects_alloc_page_count + 
            ISNULL(SUM(ts.user_objects_alloc_page_count),0)
                AS [user_alloc_page_count]
        ,ss.user_objects_dealloc_page_count + 
            ISNULL(SUM(ts.user_objects_dealloc_page_count),0)
                AS [user_dealloc_page_count]
        ,ss.user_objects_deferred_dealloc_page_count
                AS [user_deferred_page_count]
        ,ss.internal_objects_alloc_page_count + 
            ISNULL(SUM(ts.internal_objects_alloc_page_count),0)
                AS [internal_alloc_page_count]
        ,ss.internal_objects_dealloc_page_count + 
            ISNULL(SUM(ts.internal_objects_dealloc_page_count),0)
                AS [internal_dealloc_page_count]
    FROM
        sys.dm_db_session_space_usage ss WITH (NOLOCK) LEFT JOIN
            sys.dm_db_task_space_usage ts WITH (NOLOCK) ON
                ss.session_id = ts.session_id
    GROUP BY
        ss.session_id
        ,ss.user_objects_alloc_page_count 
        ,ss.user_objects_dealloc_page_count 
        ,ss.internal_objects_alloc_page_count 
        ,ss.internal_objects_dealloc_page_count
        ,ss.user_objects_deferred_dealloc_page_count
)
,SpaceUsage
AS
(
    SELECT
        session_id
        ,CONVERT(DECIMAL(12,3),([user_alloc_page_count] - [user_dealloc_page_count]) / 128.)
            AS [user_used_mb]
        ,CONVERT(DECIMAL(12,3),([internal_alloc_page_count] - [internal_dealloc_page_count]) / 128.)
            AS [internal_used_mb]
        ,CONVERT(DECIMAL(12,3),user_deferred_page_count / 128.)
            AS [user_deferred_used_mb]
    FROM
        SpaceUsagePages
)
SELECT
    su.session_id
    ,su.user_used_mb
    ,su.internal_used_mb
    ,su.user_deferred_used_mb
    ,su.user_used_mb + su.internal_used_mb AS [space_used_mb]	
    ,es.open_transaction_count
    ,es.login_time
    ,es.original_login_name
    ,es.host_name
    ,es.program_name
    ,er.status as [request_status]
    ,er.start_time
    ,CONVERT(DECIMAL(21,3),er.total_elapsed_time / 1000.) AS [duration]
    ,er.cpu_time
    ,ib.event_info as [buffer]
    ,er.wait_type
    ,er.wait_time
    ,er.wait_resource
    ,er.blocking_session_id
FROM 
    SpaceUsage su  
        LEFT JOIN sys.dm_exec_requests er WITH (NOLOCK) ON
            su.session_id = er.session_id
        LEFT JOIN sys.dm_exec_sessions es WITH (NOLOCK) ON
            su.session_id = es.session_id
        OUTER APPLY 
            sys.dm_exec_input_buffer(es.session_id, er.request_id) ib
WHERE 
     su.user_used_mb + su.internal_used_mb >= 50
ORDER BY
    [space_used_mb] DESC
OPTION (RECOMPILE);

**Analyzing version store usage per-database** 

SQL Server 2016 SP2 and above

In [ ]:
SELECT
    DB_NAME(database_id) AS [DB]
    ,database_id
    ,reserved_page_count
    ,CONVERT(DECIMAL(12,3),reserved_space_kb / 1024.)
        AS [Reserved Space (MB)]
FROM 
    sys.dm_tran_version_store_space_usage WITH (NOLOCK)
OPTION (MAXDOP 1, RECOMPILE);

**Analyzing version store usage per-database** 

Prior than SQL Server 2016. Results are less accurate.

In [ ]:
SELECT 
    DB_NAME(database_id) AS [DB]
    ,database_id
    ,CONVERT(DECIMAL(12,3),
        SUM(record_length_first_part_in_bytes + 
            record_length_second_part_in_bytes) / 1024. / 1024.
    ) AS [Version Store (MB)]
FROM
    sys.dm_tran_version_store WITH (NOLOCK)
GROUP BY
    database_id
OPTION (MAXDOP 1, RECOMPILE);


**Tracking tempdb spills**

Create xEvent Session

In [ ]:
IF EXISTS (SELECT * FROM sys.server_event_sessions WHERE name = 'Spills')
	DROP EVENT SESSION Spills ON SERVER;
GO

CREATE EVENT SESSION [Spills] 
ON SERVER 
ADD EVENT sqlserver.hash_warning
(
    ACTION
    (
        sqlserver.database_id
        ,sqlserver.plan_handle
        ,sqlserver.session_id
        ,sqlserver.sql_text
        ,sqlserver.query_hash
        ,sqlserver.query_plan_hash
    )
    WHERE ([sqlserver].[is_system]=0)
),
ADD EVENT sqlserver.sort_warning
(
    ACTION
    (
        sqlserver.database_id
        ,sqlserver.plan_handle
        ,sqlserver.session_id
        ,sqlserver.sql_text
        ,sqlserver.query_hash
        ,sqlserver.query_plan_hash
    )
    WHERE ([sqlserver].[is_system]=0)
), 
ADD EVENT sqlserver.exchange_spill
(
    ACTION
    (
        sqlserver.database_id
        ,sqlserver.plan_handle
        ,sqlserver.session_id
        ,sqlserver.sql_text
        ,sqlserver.query_hash
        ,sqlserver.query_plan_hash
    )
    WHERE ([sqlserver].[is_system]=0)
)
ADD TARGET package0.ring_buffer;
GO

Start xEvent Session

In [ ]:
ALTER EVENT SESSION [Spills] 
ON SERVER 
STATE = START;

Analyze the results

In [ ]:
IF OBJECT_ID('tempdb..#tmpXML') IS NOT NULL
    DROP TABLE #tmpXML;
GO

CREATE TABLE #tmpXML
(
    EventTime DATETIME2(7) NOT NULL,
    [Event] XML
);
GO

DECLARE 
    @TargetData XML;

SELECT  
    @TargetData = CONVERT(XML,st.target_data) 
FROM  
    sys.dm_xe_sessions s WITH (NOLOCK)  
        JOIN sys.dm_xe_session_targets st WITH(NOLOCK) ON 
            s.address = st.event_session_address 
WHERE  
    s.name = 'Spills' and st.target_name = 'ring_buffer';

INSERT INTO #tmpXML(EventTime, [Event])
    SELECT 
        t.e.value('@timestamp','datetime'), t.e.query('.') 
    FROM 
        @TargetData.nodes('/RingBufferTarget/event') AS t(e); 

;WITH EventInfo
AS
(
    SELECT
        t.EventTime
        ,t.[Event].value('/event[1]/@name','sysname') AS [Event]
        ,t.[Event].value('(/event[1]/action[@name="session_id"]/value/text())[1]'
            ,'smallint') AS [Session]
        ,t.[Event].value('(/event[1]/action[@name="database_id"]/value/text())[1]'
            ,'smallint') AS [DB]
        ,t.[Event].value('(/event[1]/action[@name="sql_text"]/value/text())[1]'
            ,'nvarchar(max)') AS [SQL]
        ,t.[Event]
            .value('(/event[1]/data[@name="granted_memory_kb"]/value/text())[1]'
                ,'bigint') AS [Granted Memory (KB)]
        ,t.[Event]
            .value('(/event[1]/data[@name="used_memory_kb"]/value/text())[1]'
                ,'bigint') AS [Used Memory (KB)]
        ,t.[Event]
  .value('xs:hexBinary((/event[1]/action[@name="plan_handle"]/value/text())[1])'
                ,'varbinary(64)') AS [PlanHandle]
        ,t.[Event].value('(/event[1]/action[@name="query_hash"]/value/text())[1]'
            ,'nvarchar(64)') AS [QueryHash]
        ,t.[Event]
            .value('(/event[1]/action[@name="query_plan_hash"]/value/text())[1]'
                ,'nvarchar(64)') AS [QueryPlanHash]
FROM
    #tmpXML t
)
SELECT
    ei.*, qp.query_plan
FROM
    EventInfo ei 
        OUTER APPLY sys.dm_exec_query_plan(ei.PlanHandle) qp
OPTION (RECOMPILE, MAXDOP 1); 

Stop xEvent session

In [ ]:
ALTER EVENT SESSION [Spills] 
ON SERVER 
STATE = STOP;

**Counting tempdb spills**

Create xEvent Session

In [ ]:
IF EXISTS (SELECT * FROM sys.server_event_sessions WHERE name = 'Spill_Count')
	DROP EVENT SESSION Spill_Count ON SERVER;
GO

CREATE EVENT SESSION [Spill_Count] 
ON SERVER 
ADD EVENT sqlserver.exchange_spill,
ADD EVENT sqlserver.hash_warning,
ADD EVENT sqlserver.sort_warning
ADD TARGET package0.event_counter;

Start xEvent Session

In [ ]:
ALTER EVENT SESSION [Spill_Count] 
ON SERVER 
STATE = START;

Analyze the results

In [ ]:
DECLARE 
    @TargetData XML

SELECT 
    @TargetData = CONVERT(XML,st.target_data) 
FROM 
    sys.dm_xe_sessions s WITH (NOLOCK) 
        JOIN sys.dm_xe_session_targets st WITH(NOLOCK) ON
            s.address = st.event_session_address
WHERE 
    s.name = 'Spill_Count' and st.target_name = 'event_counter';

;WITH EventInfo
AS
(
    SELECT
        t.e.value('@name','sysname') AS [Event] 
        ,t.e.value('@count','bigint') AS [Count] 
    FROM
        @TargetData.nodes
             ('/CounterTarget/Packages/Package[@name="sqlserver"]/Event') 
                AS t(e)
) 
SELECT [Event], [Count] 
FROM EventInfo
OPTION (RECOMPILE, MAXDOP 1); 


Stop xEvent session

In [ ]:
ALTER EVENT SESSION [Spill_Count] 
ON SERVER 
STATE = STOP;